<a href="https://colab.research.google.com/github/moussaley/Donnee/blob/main/Projet_G%C3%A9omatique_avanc%C3%A9e__ABOU_TANKO_Moussa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visualisation des données des incendies dans la zone prométhée (France)

### Installations des librairies nécéssaires au fonctionnement de l'application

In [1]:
!pip install -q dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 45.5 MB/s eta 0:00:00


In [2]:
!pip install jupyter-dash


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.7 MB/s eta 0:00:00


In [3]:
!pip install pandas

In [4]:
! pip install geopandas

In [5]:
! pip install dash_bootstrap_components

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 kB 4.3 MB/s eta 0:00:00


In [6]:
!pip install folium

In [7]:
!pip install contextily

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 28.5 MB/s eta 0:00:00


### Import des librairies nécéssaires pour lancer le code


In [8]:
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
import plotly.express as px
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go
import dash_bootstrap_components as dbc
from jupyter_dash import JupyterDash
import contextily as ctx


### Connexion au drive


In [9]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


### Récupération des données à partir de mon repository github, néttoyage et création des nouvelles colonnes

In [13]:
# Récupération des données à partir de mon repository github
mes_incendies = pd.read_csv('https://raw.githubusercontent.com/moussaley/Donnee/main/incendies%20prom.csv',  delimiter = ';', index_col = 0)
mes_incendies['Mois'] = pd.DatetimeIndex(mes_incendies['Alerte']).month # Extraire les mois
mes_incendies['Heure'] = pd.DatetimeIndex(mes_incendies['Alerte']).hour # Extraire l'heure
mes_incendies['Surface en ha'] = (mes_incendies['Surface parcourue (m2)'])/10000
Incendies_sansNul = mes_incendies.dropna(axis=0)
total_incendies = Incendies_sansNul.shape[0]
#print(total_incendies)
numero_departements = Incendies_sansNul['Département'].unique().tolist()
numero_departements.sort()
nom_depatements = ["ALPES-DE-HAUTE-PROVENCE","HAUTES-ALPES","ALPES-MARITIMES","ARDECHE","AUDE","BOUCHES-DU-RHONES",
                   "DROME","CORSE-DU-SUD","HAUTE-CORSE","GARD","HERAULT","LOZERE","PYRENEES-ORIENTALES","VAR","VAUCLUSE"]
departements = dict(zip(numero_departements, nom_depatements))
#print(departements)

annees_incendies = Incendies_sansNul.reset_index()['Année'].unique().tolist()
#print(annees_incendies)
# Pivot tables declaration
surface_brulee_par_an = pd.pivot_table(Incendies_sansNul, values='Surface en ha',index=['Année'],aggfunc=np.sum,fill_value=0)

Nbr_Incendies_par_an = pd.pivot_table(Incendies_sansNul, values='Alerte',index=['Année'],aggfunc='count',fill_value=0)

surface_brulee_par_dep = pd.pivot_table(Incendies_sansNul, values='Surface en ha',index=['Département'],aggfunc=np.sum,
                                fill_value=0).sort_values(by='Surface en ha', ascending=False)

Nbr_incendies_par_dep = pd.pivot_table(Incendies_sansNul, values='Alerte',index=['Département'],aggfunc='count',
                                fill_value=0).sort_values(by='Alerte', ascending=False)


In [14]:
surface_brulee_par_dep_tab = surface_brulee_par_dep.reset_index()
surface_brulee_par_dep_tab

,Département,Surface en ha
0,83,83931.8710
1,2B,61017.7399
2,6,54001.1821
3,13,51960.6796
4,66,51370.6139
5,30,43960.2816
6,7,41706.0245
7,34,38591.7092
8,11,36516.0711
9,2A,24211.2105


In [15]:
Nbr_incendies_par_dep_tab = Nbr_incendies_par_dep.reset_index()
Nbr_incendies_par_dep_tab

,Département,Alerte
0,83,13527
1,13,7758
2,6,7525
3,7,7462
4,30,5890
5,34,5660
6,2A,5087
7,2B,4403
8,66,4023
9,11,3577


### Calculs pour les statitisques

In [16]:
bar_chart_surface_brulee_par_dep = px.bar(surface_brulee_par_dep,x=surface_brulee_par_dep.index,y=surface_brulee_par_dep.columns,
                  color_discrete_sequence = ['#e67e22']*len(surface_brulee_par_dep),template= 'plotly_white')
bar_chart_surface_brulee_par_dep.update_layout(title_text= "Surface en ha brulée par département",title_x=0.5,showlegend=False)
bar_chart_Nbr_incendies_par_dep = px.bar(Nbr_incendies_par_dep,x=Nbr_incendies_par_dep.index,y=Nbr_incendies_par_dep.columns,
                   template= 'plotly_white', color_discrete_sequence = ['#e67e22']*len(surface_brulee_par_dep))
bar_chart_Nbr_incendies_par_dep.update_layout(title_text="Incendies par département",title_x=0.5,showlegend=False)

line_chart_surface_brulee_par_an = px.line(surface_brulee_par_an,x=surface_brulee_par_an.index,y=surface_brulee_par_an.columns,
                   color_discrete_sequence = ['#e67e22']*len(surface_brulee_par_an),template= 'plotly_white')
line_chart_surface_brulee_par_an.update_layout(title_text= "Surface en ha brulée par an",title_x=0.5,showlegend=False)

line_chart_Nbr_incendies_par_an = px.line(Nbr_Incendies_par_an,x=Nbr_Incendies_par_an.index,y=Nbr_Incendies_par_an.columns,
                 color_discrete_sequence=['#e67e22']*len(Nbr_Incendies_par_an),template= 'plotly_white')
line_chart_Nbr_incendies_par_an.update_layout(title_text= "Nombre d'incendies par an",title_x=0.5,showlegend=False)

### Récupération des données géographiques à partir de mon github

In [20]:
shapefile = gpd.read_file('https://raw.githubusercontent.com/moussaley/JSON/main/fichier_incendies.geojson')
shapefile

,code,nom,geometry
0,04,Alpes-de-Haute-Provence,"POLYGON ((5.67604 44.19143, 5.67817 44.19051, ..."
1,05,Hautes-Alpes,"POLYGON ((6.26057 45.12685, 6.26417 45.12641, ..."
2,06,Alpes-Maritimes,"MULTIPOLYGON (((7.06711 43.51365, 7.06665 43.5..."
3,2A,Corse-du-Sud,"MULTIPOLYGON (((8.58640 41.87389, 8.58667 41.8..."
4,2B,Haute-Corse,"MULTIPOLYGON (((9.40875 43.02232, 9.40662 43.0..."
5,07,Ardèche,"POLYGON ((4.48313 45.23645, 4.48790 45.23218, ..."
6,11,Aude,"POLYGON ((1.68842 43.27355, 1.69029 43.27437, ..."
7,13,Bouches-du-Rhône,"MULTIPOLYGON (((5.39670 43.17300, 5.39314 43.1..."
8,26,Drôme,"POLYGON ((4.80049 45.29836, 4.80552 45.29935, ..."
9,30,Gard,"POLYGON ((3.37365 44.17076, 3.38309 44.17041, ..."


In [21]:
 bound = shapefile[shapefile['nom'] == 'Var'].geometry.total_bounds
 X = (bound[0] + bound[2])/2
 Y  = (bound[1] + bound[3])/2
 X,Y

(6.29485, 43.395285)

### Calcul des coordonnées du point sur lequel centré les cartes

In [22]:
bound = shapefile.total_bounds
X  = (bound[0] + bound[2])/2
Y  = (bound[1] + bound[3])/2
X,Y

(5.6241900000000005, 43.34985)

### Définition des variables cartes

In [23]:
map_surface_brulee = folium.Map(location=[Y,X], zoom_start=6)
folium.Choropleth(
    geo_data=shapefile.to_json(),
    name='Surface brulée',
    data=surface_brulee_par_dep_tab,
    columns=['Département', 'Surface en ha'],
    key_on='feature.properties.code',
    fill_color='OrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Surface brulée en HA',
).add_to(map_surface_brulee)
folium.LayerControl().add_to(map_surface_brulee)

map_incendies = folium.Map(location=[Y,X], zoom_start=6)
folium.Choropleth(
    geo_data=shapefile.to_json(),
    name='Incendies',
    data=Nbr_incendies_par_dep_tab,
    columns=['Département', 'Alerte'],
    key_on='feature.properties.code',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Nombre d\'Alertes par départements"
).add_to(map_incendies)
folium.LayerControl().add_to(map_incendies)

### Création des fichiers HTML pour l'affichage des cartes

In [24]:
map_surface_brulee.save("/content/drive/MyDrive/map_surface.html")
map_incendies.save("/content/drive/MyDrive/map_incedies.html")

### Calcul de la surface totale brulée dans tous les départements

In [25]:
#Calculer la surface totale brulée pour tous les départements
surface_total = 0
for i , row in surface_brulee_par_dep_tab.iterrows() :
  #print(row['Surface en ha'])
  surface_total += row['Surface en ha']
surface_total

519793.92809999996

### Création et design du tableau de bord

In [26]:
import os
# Dash app development part
app = JupyterDash(__name__,external_stylesheets=[dbc.themes.MORPH])
app.title = "Incendies de forêts dans la zone promethée : 1973 - 2022"

app.head = [html.Link(href='assets/favicon.ico',rel='icon')],


app.layout = html.Div(className = "row", children=[
    html.H1(children="Incendies de forêts dans la zone prométhée : 1973 - 2022", style={'textAlign': 'center'}),

    html.H4(children="Réalisé par ABOU TANKO Moussa", style={'textAlign': 'center'}),

    dcc.Tabs(id='tabs', value='tab-1', children=[

        dcc.Tab(label='Statistiques générales', value='tab-1', children=[
            html.Div(
            className="row",
            children=[
                html.Div(
                    className="col-6",
                    children=[
                        html.Div(
                            className="card",
                            children=[
                                html.Div(className="card-title",
                                         children=  " Nombre d\'incendies total",
                                         style={"font-size": "2rem",
                                                'text-align': 'center',
                                                'color': '#e67e22'},),
                                html.Div(className="card-body",
                                         children= str(total_incendies),
                                       style={"font-size": "3rem",
                                              'text-align': 'center',
                                              'color': '#e67e22'}  ),
                            ],
                        )
                    ],
                    style={
                  'margin': '5px 0px 5px 0px'
                   },
                ),
               html.Div(
                    className="col-6",
                    children=[
                        html.Div(
                            className="card",
                            children=[
                                html.Div(className="card-title",
                                         children=  "Surface totale brulée en ha",
                                         style={"font-size": "2rem",
                                                'text-align': 'center',
                                                'color': '#e67e22'},),
                                html.Div(className="card-body",
                                         children= str(round(surface_total,2)),
                                       style={"font-size": "3rem",
                                              'text-align': 'center',
                                              'color': '#e67e22'}  ),
                            ],
                        )
                    ],
                    style={
                  'margin': '5px 0px 5px 0px'
                   },
                ),
            ],
        ),
    dcc.Graph(
        id="graph1",
        style={
            'display': 'inline-block',
            'vertical-align': 'top',
            'width': '49%',
            'margin': '10px 5px 0px 5px'
        },
        figure=bar_chart_surface_brulee_par_dep
    ),
dcc.Graph(
        id="graph2",
        style={
            'display': 'inline-block',
            'vertical-align': 'top',
            'width': '49%',
            'margin': '10px 5px 0px 5px'

        },
        figure=line_chart_Nbr_incendies_par_an
    ),

    html.Hr(),
dcc.Graph(
        id="graph3",
    style={
        'display': 'inline-block',
        'vertical-align': 'top',
        'width': '49%',
        'margin': '0px 5px 0px 5px'
    },
        figure=line_chart_surface_brulee_par_an
    ),
    dcc.Graph(
        id="graph4",
        style={
            'display': 'inline-block',
            'vertical-align': 'top',
            'width': '49%',
            'margin': '0px 5px 0px 5px'
        },
        figure=bar_chart_Nbr_incendies_par_dep ),
        ]),

        dcc.Tab(label='Statistiques par département', value='tab-3', children=[

        dcc.Dropdown(list(departements.values()), id='pandas-dropdown-2',
                    placeholder= "Choisir un département",
                    className="one-third column",
                      style={
                'margin': '5px 5px'
            }),
         html.Div([
             html.Div(
                 html.Iframe(id = "map_surface", srcDoc= open('/content/drive/MyDrive/map_surface.html','r').read(), width="95%",height='600'),
                 style={'display': 'inline-block', 'width': '49%','margin': '5px 5px'}
             ),
               html.Div(
                 html.Iframe(id = "map_incedies", srcDoc= open('/content/drive/MyDrive/map_incedies.html','r').read(), width="95%",height='600'),
                 style={'display': 'inline-block', 'width': '49%','margin': '5px 5px'}
             )
            ])

        ]),
        dcc.Tab(label='Statistiques par année', value='tab-2', children=[
            html.Div([
                  dcc.Dropdown(annees_incendies, id='dates-dropdown',
                  placeholder= "Choisir une année ",
                  style={
                    'margin': '5px 5px'
                  }),
                ], className='six columns'),
          html.Div([
             html.Div(
                 html.Iframe(id = "map_surface2", srcDoc= open('/content/drive/MyDrive/map_surface.html','r').read(), width="95%",height='600'),
                 style={'display': 'inline-block', 'width': '49%','margin': '5px 5px'}
             ),
               html.Div(
                 html.Iframe(id = "map_incedies2", srcDoc= open('/content/drive/MyDrive/map_incedies.html','r').read(), width="95%",height='600'),
                 style={'display': 'inline-block', 'width': '49%','margin': '5px 5px'}
             )
            ])
        ]),
    ])
])

@app.callback(
    Output('map_surface', 'children'), [Input('pandas-dropdown-2', 'value')])

def update_map(departement_choisi):

    """
    bound = shapefile[shapefile['nom'] == departement_choisi]
    boundExtent = shapefile[shapefile['nom'] == departement_choisi].geometry.total_bounds
    X = (boundExten[0] + boundExten[2])/2
    Y  = (boundExten[1] + boundExten[3])/2
    map_surface_brulee = folium.Map(location=[Y,X], zoom_start=6)
    folium.Choropleth(
      geo_data=bound.to_json(),
      name='Surface brulée',
      data=surface_brulee_par_dep_tab,
      columns=['Département', 'Surface en ha'],
      key_on='feature.properties.code',
      fill_color='OrRd',
      fill_opacity=0.7,
      line_opacity=0.2,
      legend_name='Surface brulée en HA',
      ).add_to(map_surface_brulee)
    folium.LayerControl().add_to(map_surface_brulee)

    fichier = "/content/drive/MyDrive/map_surface_brulee_" + departement_choisi + ".html"

    if not os.path.isfile(fichier):
          map_surface_brulee.save(fichier)
    return  html.Iframe( srcDoc= open(fichier,'r').read(), width="95%",height='600')
    """


@app.callback(
    Output('map_surface2', 'children'),Input('dates-dropdown', 'value')
)
def update_output(value):
    return f'You have selected {value}'


# Pour voir l'application dans un nouvel onglet décommentez ces deux lignes de code et commentez les 3 dernières
if __name__ == '__main__':
  app.run_server(debug=True)


#app.css.config.serve_locally=True
#app.scripts.config.serve_locally=True
#app.run_server(mode = 'inline', port = 8030)


/usr/local/lib/python3.10/dist-packages/dash/dash.py:556: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



<IPython.core.display.Javascript object>

Dash app running on:


<IPython.core.display.Javascript object>